In [4]:
import numpy as np
import pandas as pd
from main import LogisticRegression,accuracy
from sklearn.datasets import make_classification

# Logit

In [5]:
np.random.seed(42)
# Generate  dataset
X, y = make_classification(
    n_samples=1000,      # Number of samples
    n_features=4,       # Number of features
    n_redundant=0,      
    n_clusters_per_class=1,
    flip_y=0.1,         # Add label noise
    class_sep=1.0,      # Class separation
)

# Add Gaussian noise to the features
noise = np.random.normal(0, 0.5, X.shape)
X = X + noise

In [ ]:
#with constant
model=LogisticRegression(eps=0.001,alpha=0.01)
model.train(np.hstack([np.ones((X.shape[0],1)),X]),y)
logit_predictions=model.predict(np.hstack([np.ones((X.shape[0],1)),X]),thr=0.5)
accuracy(logit_predictions,y)

np.float64(0.827)

In [9]:
#without
model=LogisticRegression(eps=1e-5,alpha=0.1)
model.train(X,y)
logit_predictions=model.predict(X,thr=0.5)
np.mean(logit_predictions.flatten() == y.flatten())
accuracy(logit_predictions,y)

np.float64(0.825)

here we just used simple gradient ascent to maximise LL, other allgorithms are presented in ml_parametric

# MLP

In [10]:
from main import NN_multi_layer_one_width

In [12]:
i=1
model=NN_multi_layer_one_width(nb_layers=i,max_iter=1000,thr=1e-5,alpha=0.1)
model.train(X,y)
nn_predictions=model.predict(X) 
print(i,"layerss",accuracy(nn_predictions,y))

‖∇b_first‖ = 0.02979515275601961
‖∇b_first‖ = 0.02893390482779168
‖∇b_first‖ = 0.028111942108775462
‖∇b_first‖ = 0.027328412670119744
‖∇b_first‖ = 0.026582427492881335
‖∇b_first‖ = 0.025873066210851215
‖∇b_first‖ = 0.025199382751513406
‖∇b_first‖ = 0.0245604108177938
‖∇b_first‖ = 0.023955169159915357
‖∇b_first‖ = 0.023382666593795242
‖∇b_first‖ = 0.022841906729783378
‖∇b_first‖ = 0.02233189238295058
‖∇b_first‖ = 0.02185162964341068
‖∇b_first‖ = 0.02140013159214242
‖∇b_first‖ = 0.020976421654336525
‖∇b_first‖ = 0.020579536588323705
‖∇b_first‖ = 0.020208529113565847
‖∇b_first‖ = 0.019862470185959276
‖∇b_first‖ = 0.019540450932780895
‖∇b_first‖ = 0.019241584262997827
‖∇b_first‖ = 0.018965006171369297
‖∇b_first‖ = 0.018709876756827698
‖∇b_first‖ = 0.018475380977071142
‖∇b_first‖ = 0.018260729162184915
‖∇b_first‖ = 0.018065157310492515
‖∇b_first‖ = 0.017887927189776717
‖∇b_first‖ = 0.01772832626657412
‖∇b_first‖ = 0.017585667485494096
‖∇b_first‖ = 0.017459288919506245
‖∇b_first‖ = 0.0173485

gradients more or less ok in L2

In [14]:
i=3
model=NN_multi_layer_one_width(nb_layers=i,max_iter=1000,thr=1e-5,alpha=0.1)
model.train(X,y)
nn_predictions=model.predict(X) 
print(i,"layerss",accuracy(nn_predictions,y))

‖∇b_first‖ = 0.0031184143472995146
‖∇b_first‖ = 0.0030096040695633645
‖∇b_first‖ = 0.002904963018999877
‖∇b_first‖ = 0.0028043637698571705
‖∇b_first‖ = 0.002707677757962601
‖∇b_first‖ = 0.0026147759133653913
‖∇b_first‖ = 0.002525529249767913
‖∇b_first‖ = 0.0024398094080033024
‖∇b_first‖ = 0.002357489151725247
‖∇b_first‖ = 0.0022784428143322348
‖∇b_first‖ = 0.002202546696937029
‖∇b_first‖ = 0.0021296794179020977
‖∇b_first‖ = 0.0020597222150846974
‖∇b_first‖ = 0.001992559202467609
‖∇b_first‖ = 0.0019280775832919452
‖∇b_first‖ = 0.0018661678221589214
‖∇b_first‖ = 0.0018067237788317043
‖∇b_first‖ = 0.0017496428066523576
‖∇b_first‖ = 0.0016948258185995682
‖∇b_first‖ = 0.0016421773240579338
‖∇b_first‖ = 0.0015916054393577378
‖∇b_first‖ = 0.0015430218750837649
‖∇b_first‖ = 0.0014963419030513767
‖∇b_first‖ = 0.0014514843057154088
‖∇b_first‖ = 0.0014083713106206998
‖∇b_first‖ = 0.0013669285123282143
‖∇b_first‖ = 0.0013270847840647479
‖∇b_first‖ = 0.0012887721811517606
‖∇b_first‖ = 0.00125192583

problem as algo did not learn, so try to work on initialisation or other activation fct (larger alpha dont help here) 